niet alle packages zijn nodig, ik heb gewoon geplakt vanuit het huidige notebook

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os

import cv2 
# pip install .whl file from https://www.lfd.uci.edu/~gohlke/pythonlibs/#opencv
# pip install numpy --upgrade if numpy.multiarray error

import time
import math
import pandas as pd

In [ ]:
def to_coordlist(input_file):
    """gets the coordinates from the coordinate file 
    and turns this into a list with coodinates"""
    people_per_file = []
    for path, subdirs, files in os.walk(input_file):
        for filename in files:
            coord_path = os.path.join(path, filename)
            with open(coord_path) as f:
                people_per_file.append(json.load(f)['people'])
    
    plottables_per_file = []
    coord_list = []
    for file in people_per_file:
        plot_lines = []
        plot_coords = []
        plottables = {}

        coords = {}
        for i, person in enumerate(file):
            coords[i] = np.array([[x,-y,z] for x,y,z in np.reshape(person['pose_keypoints'], (18,3))]) # constraint on confidence

        for person in coords.keys():
            try:
                plot_coords = plot_coords + list(coords[person][~(coords[person]==0).any(axis=1)])
            except:
                continue

            coord_dict = {key:value for key, value in  dict(enumerate(coords[person][:, :2])).items() if 0 not in value}


    #thist makes sure to only pick the relevant person. purely manual, not automatically the most interesting one        
            if person == 1:
                coord_list.append(coord_dict)
    return coord_list

In [ ]:
def to_coord_df(coord_list):
    """gets a list of coordinates and turns this into a dataframe"""
    coord_df = pd.DataFrame(coord_list)

    coord_df.columns = ['Nose', 'Neck', 'Right Shoulder', 'Right Elbow', 'Right Hand',
                  'Left Shoulder', 'Left Elbow', 'Left Hand',
                  'Right Hip', 'Right Knee', 'Right Foot', 'Left Hip', 'Left Knee', 'Left Foot',
                  'Right Eye', 'Left Eye', 'Right Ear', 'Left Ear']

    #add the frame number
    coord_df['Frame'] = coord_df.index

    #melt the dataframe to get the locations in one row
    coord_df = pd.melt(coord_df, id_vars = 'Frame', var_name = 'Point', value_name = 'Location')

    #split up the coordinates and put them into separate columns
    coord_df['Split'] = coord_df.Location.apply(lambda x: str(x).split('  '))
    coord_df['x'] = coord_df.Location.str.get(0)
    coord_df['y'] = coord_df.Location.str.get(1)

    #delete irrelevant columns
    del coord_df['Split']
    del coord_df['Location']

    coord_df.to_csv('coordinate_df')

    #coord_df = coord_df[coord_df['x'] <= 1780]

    return coord_df

In [ ]:
def to_feature_df(coord_df):
    """gets a dataframe of coordinates and turns this into features. 
    In this case, the standard deviation of movement both horizontally as well as vertically"""
    coord_df['video'] = 1
    
    x_df = coord_df.pivot_table(index='video', columns = 'Point', values = 'x', aggfunc = np.std)
    y_df = coord_df.pivot_table(index='video', columns = 'Point', values = 'y', aggfunc = np.std)
    
    x_df.columns = [str(col) + '_x' for col in x_df.columns]
    y_df.columns = [str(col) + '_y' for col in y_df.columns]
    
    x_df['video'] = x_df.index
    y_df['video'] = y_df.index
    
    std_df = pd.merge(x_df, y_df, on='video', how='outer')
    
    return std_df

In [ ]:
input_file = r'C:\Users\20175863\Documents\JADS\DEiA2\openpose-1.2.1-win64-binaries\coordinates'

coordlist = to_coordlist(input_file)
coord_df = to_coord_df(coordlist)
feature_df = to_feature_df(coord_df)
feature_df